In [1]:
import numpy as np
import pandas as pd
from os import path

In [10]:
from sapiens import predict_sequence_embedding

In [3]:
DATA_DIR = "../../data/"

In [5]:
chen_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_data.csv"), index_col=0)
chen_data.head(5)

,Antibody_ID,heavy,light,Y
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0


In [54]:
chen_data_short = chen_data[chen_data["heavy"].str.len() <= 140]
print(len(chen_data_short))
chen_data_short.head()

1562


,Antibody_ID,heavy,light,Y
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0


In [48]:
def embed_sequence(row):
    heavy = predict_sequence_embedding(
        row["heavy"], 
        "H", 
        layer=-1
    )
    light = predict_sequence_embedding(
        row["light"], 
        "L", 
        layer=-1
    )
    
    embedding = np.concatenate([heavy, light])
    return embedding

In [28]:
embed_sequence(chen_data.loc[1618])

In [55]:
embeddings = pd.DataFrame(chen_data_short.apply(embed_sequence, axis=1).to_list())
embeddings.insert(0, "Ab_ID", chen_data_short["Antibody_ID"])
embeddings.columns = embeddings.columns.astype(str)
#embeddings.to_feather(path.join(DATA_DIR, "chen/embeddings/sapiens/sapiens_chen_embeddings.ftr"))
embeddings.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,12e8,-0.134161,0.075484,0.377024,0.264288,0.132392,-0.075669,-0.014079,-0.373292,-0.028935,...,-0.551750,-0.657874,0.348789,0.660802,0.240210,-1.159543,-0.089259,-0.296646,-0.692070,-0.279281
1,15c8,-0.188603,0.159150,0.352639,0.196398,0.050981,-0.047057,0.092863,-0.381236,-0.021599,...,0.052712,0.733469,0.215582,0.243388,1.013782,-1.221909,-0.350015,-1.193482,-0.844036,0.096152
2,1a0q,-0.216819,0.059556,0.403906,0.228114,0.103585,-0.018792,0.021493,-0.419275,-0.069553,...,-0.303439,-0.594165,0.028933,0.760616,0.311758,-1.261243,-0.089060,-0.164134,-0.463464,-0.128049
3,1a14,-0.156908,0.098237,0.386578,0.178501,0.140005,-0.026292,0.077186,-0.382542,-0.069449,...,-0.458109,-0.684457,0.094104,0.968093,0.299307,-1.249306,-0.307804,-0.359713,-0.618798,-0.211896
4,1a2y,0.242810,-0.101020,0.334529,-0.154067,0.018751,-0.229859,0.122908,-0.395358,0.087788,...,-0.505897,-0.374633,0.219703,0.776408,0.592473,-1.179988,-0.285079,-0.164498,-0.650573,-0.127861


In [56]:
embeddings["Y"] = chen_data_short["Y"]
embeddings.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,Y
0,12e8,-0.134161,0.075484,0.377024,0.264288,0.132392,-0.075669,-0.014079,-0.373292,-0.028935,...,-0.657874,0.348789,0.660802,0.240210,-1.159543,-0.089259,-0.296646,-0.692070,-0.279281,0.0
1,15c8,-0.188603,0.159150,0.352639,0.196398,0.050981,-0.047057,0.092863,-0.381236,-0.021599,...,0.733469,0.215582,0.243388,1.013782,-1.221909,-0.350015,-1.193482,-0.844036,0.096152,0.0
2,1a0q,-0.216819,0.059556,0.403906,0.228114,0.103585,-0.018792,0.021493,-0.419275,-0.069553,...,-0.594165,0.028933,0.760616,0.311758,-1.261243,-0.089060,-0.164134,-0.463464,-0.128049,1.0
3,1a14,-0.156908,0.098237,0.386578,0.178501,0.140005,-0.026292,0.077186,-0.382542,-0.069449,...,-0.684457,0.094104,0.968093,0.299307,-1.249306,-0.307804,-0.359713,-0.618798,-0.211896,0.0
4,1a2y,0.242810,-0.101020,0.334529,-0.154067,0.018751,-0.229859,0.122908,-0.395358,0.087788,...,-0.374633,0.219703,0.776408,0.592473,-1.179988,-0.285079,-0.164498,-0.650573,-0.127861,0.0


In [57]:
embeddings.to_feather(path.join(DATA_DIR, "chen/embeddings/sapiens/sapiens_chen_embeddings.ftr"))

In [46]:
chen_data[chen_data["heavy"].str.len() > 143]
indices = chen_data[chen_data["heavy"].str.len() > 140].index

In [52]:
chen_data[chen_data["heavy"].str.len() == 140]

,Antibody_ID,heavy,light,Y
1008,3u1s,QVQLVQSGAEVKKPGSSVKVSCKASGNSFSNHDVHWVRQATGQGLE...,EVVITQSPLFLPVTPGEAASLSCKCSHSLQHSTGANYLAWYLQRPG...,0


In [53]:
embed_sequence(chen_data.loc[1008])

array([-0.28252992,  0.07246271,  0.29181525,  0.11483017,  0.12999843,
       -0.07059711, -0.22013626, -0.32268146, -0.05911254,  0.0290522 ,
       -0.25872204, -0.11831281,  0.17609318,  0.0963475 ,  0.01062322,
       -0.13352418, -0.04778805,  0.24351208,  0.07833483,  0.02363428,
       -0.02278778, -0.33563596,  0.02135167,  0.04609078, -0.00578414,
       -0.3145087 , -0.3899804 ,  0.03675899, -0.00855669,  0.25341177,
        0.03331763, -0.02998232, -0.12642251,  0.13882753, -0.13271187,
        0.08363034, -0.26176643,  0.05960616,  0.4383062 , -0.28734007,
        0.2759207 ,  0.12156837,  0.01480813, -0.05221846, -0.36704844,
       -0.13116908,  0.15756232, -0.2167949 , -0.14026366, -0.13674779,
        0.2026569 , -0.28815833, -0.19025286, -0.23410955,  0.03381168,
       -0.14780466,  0.05004765, -0.14252734,  0.0545227 , -0.3362012 ,
       -0.26914197, -0.14329153,  0.01887868,  0.21291602,  0.28161976,
       -0.34518328,  0.03528953, -0.0649538 ,  0.28079352, -0.10

In [58]:
tap_data = pd.read_csv(path.join(DATA_DIR, "tap/TAP_data.csv"))
tap_data.head()

,Antibody_ID,heavy,light,CDR_length,PSH,PPC,PNC,SFvCSP,Y
0,Abagovomab,QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLD...,DIELTQSPASLSASVGETVTITCQASENIYSYLAWHQQKQGKSPQL...,46,129.7603,0.0000,0.0000,16.32,1
1,Abituzumab,QVQLQQSGGELAKPGASVKVSCKASGYTFSSFWMHWVRQAPGQGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDISNYLAWYQQKPGKAPKL...,45,115.9106,0.0954,0.0421,-3.10,1
2,Abrilumab,QVQLVQSGAEVKKPGASVKVSCKVSGYTLSDLSIHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQQKPGKAPKL...,45,109.6995,0.0000,0.8965,-4.00,1
3,Actoxumab,QVQLVESGGGVVQPGRSLRLSCAASGFSFSNYGMHWVRQAPGKGLE...,DIQMTQSPSSVSASVGDRVTITCRASQGISSWLAWYQHKPGKAPKL...,49,112.6290,0.0000,1.1247,3.10,1
4,Adalimumab,EVQLVESGGGLVQPGRSLRLSCAASGFTFDDYAMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQGIRNYLAWYQQKPGKAPKL...,48,111.2512,0.0485,1.1364,-19.50,1


In [59]:
embeddings_tap = pd.DataFrame(tap_data.apply(embed_sequence, axis=1).to_list())
embeddings_tap.insert(0, "Ab_ID", tap_data["Antibody_ID"])
embeddings_tap.columns = embeddings_tap.columns.astype(str)
embeddings_tap.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,Abagovomab,-0.090748,0.099529,0.442706,0.176952,0.117761,-0.001945,0.023426,-0.350218,-0.185384,...,-0.620560,-0.482742,0.217774,0.850484,0.649386,-1.274149,-0.279191,-0.268616,-0.737053,-0.196778
1,Abituzumab,-0.074267,0.171551,0.474691,0.133427,0.216521,0.047407,-0.020084,-0.492360,-0.133921,...,-0.747846,-0.788665,-0.007604,0.722280,0.053256,-1.161701,-0.103081,-0.842254,-0.743219,-0.017302
2,Abrilumab,0.119964,-0.059644,0.443711,0.139779,0.575773,0.334234,-0.348093,-0.776167,0.025728,...,-0.602528,-0.751126,0.074466,0.664460,0.072910,-1.229350,-0.027697,-1.512877,-0.822899,0.290256
3,Actoxumab,0.000611,-0.089507,0.338677,0.083151,0.200531,0.330401,-0.253224,-0.555152,-0.189456,...,-0.620141,-0.764126,-0.023048,0.572022,-0.070313,-1.173917,0.022668,-1.612663,-0.839945,0.370874
4,Adalimumab,0.230531,-0.000938,0.238706,-0.071658,0.297986,0.500569,-0.130903,-0.580121,-0.344312,...,-0.594686,-0.816617,-0.204169,0.574035,-0.107883,-1.014039,0.043726,-0.813334,-0.568057,0.043446


In [60]:
embeddings_tap["Y"] = tap_data["Y"]
embeddings_tap.head()

,Ab_ID,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,Y
0,Abagovomab,-0.090748,0.099529,0.442706,0.176952,0.117761,-0.001945,0.023426,-0.350218,-0.185384,...,-0.482742,0.217774,0.850484,0.649386,-1.274149,-0.279191,-0.268616,-0.737053,-0.196778,1
1,Abituzumab,-0.074267,0.171551,0.474691,0.133427,0.216521,0.047407,-0.020084,-0.492360,-0.133921,...,-0.788665,-0.007604,0.722280,0.053256,-1.161701,-0.103081,-0.842254,-0.743219,-0.017302,1
2,Abrilumab,0.119964,-0.059644,0.443711,0.139779,0.575773,0.334234,-0.348093,-0.776167,0.025728,...,-0.751126,0.074466,0.664460,0.072910,-1.229350,-0.027697,-1.512877,-0.822899,0.290256,1
3,Actoxumab,0.000611,-0.089507,0.338677,0.083151,0.200531,0.330401,-0.253224,-0.555152,-0.189456,...,-0.764126,-0.023048,0.572022,-0.070313,-1.173917,0.022668,-1.612663,-0.839945,0.370874,1
4,Adalimumab,0.230531,-0.000938,0.238706,-0.071658,0.297986,0.500569,-0.130903,-0.580121,-0.344312,...,-0.816617,-0.204169,0.574035,-0.107883,-1.014039,0.043726,-0.813334,-0.568057,0.043446,1


In [62]:
embeddings_tap.to_feather(path.join(DATA_DIR, "tap/embeddings/sapiens/sapiens_tap_embeddings.ftr"))